In [1]:
##############
#IMPORTS
%reset

import pickle
import numpy as np
import copy 
from sklearn import decomposition
import os
import matplotlib.pyplot as plt
import yaml
from keras import models
from keras import layers
from keras import optimizers
import tensorflow as tf
from helpers import helper_functions

def load_obj(name):
    with open(name+'.pkl', 'rb') as f:
        return pickle.load(f, encoding='latin1')

# Gaussian normalization, return 0 if std is 0
def normalize(obj, mean, std):
    a = obj-mean
    b=std
    return np.divide(a, b, out=np.zeros_like(a), where=b!=0)

# load conf file, 'config'
def load_config(config_file):
    with open(config_file) as f:
        config = yaml.load(f)
    return config

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


Using TensorFlow backend.


In [4]:
# define a function to load in the necessary data, first creating the data files with the necessary delay times
def data_loader(data_dir):
    
#     print("Loading the model/conf file in from: " + results_dir)
    
#     ################
#     # Load in the conf file
#     model_filename='model.h5'




#     config=load_config(os.path.join(results_dir,'conf.yaml'))['data']
#     config_entire = load_config(os.path.join(results_dir,'conf.yaml'))


#     ###############
#     #LOAD the model in

    
#     # load model, 'loaded_model'
#     loaded_model=models.load_model(os.path.join(results_dir,model_filename))

#     loaded_model.compile(optimizer=optimizers.RMSprop(lr=.001),
#                          metrics=['mae'], loss='mse')
#     print("This is what the conf file you gave me looks like: ")
#     print(config)


    #############
#     # preprocess the data and save it based on the conf file
#     helper_functions.preprocess_data(dirname=data_dir, 
#                                     sigs_0d=config['sigs_0d'],
#                                     sigs_1d=config['sigs_1d'],
#                                     sigs_predict=config['sigs_predict'],
#                                     n_components=config['n_components'],
#                                     avg_window=config['avg_window'],
#                                     lookback=config['lookback'],
#                                     delay=config['delay'],
#                                     train_frac=config['train_frac'],
#                                     val_frac=config['val_frac'],
#                                     save_data=True, noised_signal = None)    
    

    print("Reading the newly processed data from: " + data_dir)
    
    # load raw data, 'raw_data'
    with open(os.path.join(data_dir,'final_data.pkl'), 'rb') as f: 
            raw_data=pickle.load(f, encoding='latin1')


    # load processed train data, 'train_data'
    with open(os.path.join(data_dir,'train_data.pkl'), 'rb') as f: 
            train_data=pickle.load(f, encoding='latin1')


    # load processed val data, 'val_data'
    with open(os.path.join(data_dir,'val_data.pkl'), 'rb') as f: 
            val_data=pickle.load(f, encoding='latin1')

    # load processed train target, 'train_target'
    with open(os.path.join(data_dir,'train_target.pkl'), 'rb') as f: 
            train_target=pickle.load(f, encoding='latin1')

    # load processed val target, 'val_target'
    with open(os.path.join(data_dir,'val_target.pkl'), 'rb') as f: 
            val_target=pickle.load(f, encoding='latin1')


    # load processed shot indices, a dictionary, 'shot_dict'
    with open(os.path.join(data_dir,'shot_indices.pkl'), 'rb') as f: 
            shot_dict=pickle.load(f, encoding='latin1')





    # load process true train times, 'train_real_times'
    with open(os.path.join(data_dir,'train_time.pkl'), 'rb') as f: 
            train_real_times=pickle.load(f, encoding='latin1')

    # load process true val times, 'val_real_times'
    with open(os.path.join(data_dir,'val_time.pkl'), 'rb') as f: 
            val_real_times=pickle.load(f, encoding='latin1')


    # load standard rho points, 'rho_points'
    with open(os.path.join(data_dir,'rho_standard.pkl'), 'rb') as f: 
            rho_points=pickle.load(f, encoding='latin1')


    # load means and standard deviations of data
    means = helper_functions.load_obj(data_dir+'/means')
    stds = helper_functions.load_obj(data_dir+'/stds')


    #################
    #Data organization

    #list of all the shots in sorted (depending) order based on number, 'train_shot_nums'
    train_shot_nums = shot_dict['train_shot_names']

    #list of all the shots in sorted (depending) order based on number

    val_shot_nums = shot_dict['val_shot_names']

    # list of the indices where each shot begins in the data, 'train_shot_indices'
    train_shot_indices = shot_dict['train_shot_indices']

    # list of the indices where each shot begins in the data, 'val_shot_indices'
    val_shot_indices = shot_dict['val_shot_indices']


    # list of 0D and 1D signals, 'sig_keys_0d' and 'sig_keys_1d'
    sig_keys_0d = config['sigs_0d'] #['curr_target', 'pinj']
    sig_keys_1d = config['sigs_1d'] #['e_temp']

    #print(sig_keys_0d)
    all_sigs = sig_keys_0d+sig_keys_1d




#     # delay and lookback values
#     delay = config['delay']
#     lookback = config['lookback']
    
    
    
    #compute baseline MAE for val
    val_baseline_mae=abs(val_target)
    val_avg_baseline_mae=np.mean(val_baseline_mae, axis=0)
    val_std_baseline_mae=np.std(val_baseline_mae, axis=0)
    
    
    
    #compute baseline MSE for val
    val_baseline_mse=np.square(val_target)
    val_avg_baseline_mse=np.mean(val_baseline_mse, axis=0)
    
    
    #compute baseline MAE for train
    train_baseline_mae=abs(train_target)
    train_avg_baseline_mae=np.mean(train_baseline_mae, axis=0)
    train_std_baseline_mae=np.std(train_baseline_mae, axis=0)
    
    
    
    #compute baseline MSE for train
    train_baseline_mse=np.square(train_target)
    train_avg_baseline_mse=np.mean(train_baseline_mse, axis=0)
    
    
    
    
    
    
    
    
    
    
    
    output_dict = {
    'raw_data' : raw_data,
    'train_data' : train_data,
    'val_data' : val_data,
    'train_target' : train_target,
    'val_target' : val_target,
    'shot_dict' : shot_dict,
    'loaded_model' : loaded_model,
    'train_real_times' : train_real_times,
    'val_real_times' : val_real_times,
    'rho_points' : rho_points,
    'means' : means,
    'stds' : stds,
    'train_shot_nums' : train_shot_nums,
    'val_shot_nums' : val_shot_nums,
    'train_shot_indices' : train_shot_indices,
    'val_shot_indices' : val_shot_indices,
    'sig_keys_0d' : sig_keys_0d,
    'sig_keys_1d' : sig_keys_1d,
    'all_sigs' : all_sigs,
    'delay' : delay,
    'lookback' : lookback,
    'baselines' : {
        'val_avg_baseline_mae': val_avg_baseline_mae,
        'val_avg_baseline_mse': val_avg_baseline_mse,
        'train_avg_baseline_mae': train_avg_baseline_mae,
        'train_avg_baseline_mse': train_avg_baseline_mse,  
        }
  
    
    
    }
    
    #sanity check that the data matches the delay:
    print("What the timestep window should be: " + str(delay+lookback+1))
    print("What the timestep window is: " + str(train_data.shape[1]))
    if(delay+lookback+1 != train_data.shape[1]):
        print("ABORT PROBLEM WITH THE DELAYS NOT MATCHING+++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

     #check how many timesteps there are   
    print(train_target.shape[0])   
    print("DONE=============================================================")

   
    
    
    
    return output_dict




In [6]:
data_dir='/global/homes/a/al34/new_data_files/'
# load processed train data, 'train_data'
with open(os.path.join(data_dir,'val_data.pkl'), 'rb') as f: 
        val_data=pickle.load(f, encoding='latin1')
        
        
with open(os.path.join(data_dir,'val_target.pkl'), 'rb') as f: 
        val_target=pickle.load(f, encoding='latin1')

In [15]:
val_data[:1].shape

(1, 16, 67)

In [16]:
val_target[:1].shape

(1, 65)

In [17]:
val_data[:1]

array([[[-3.39101466, -1.94930836, -1.23826798, ..., -0.90812404,
         -0.5339706 , -0.18310244],
        [-3.32364165, -1.25203185, -1.2524483 , ..., -0.80530204,
         -0.30525659, -0.02134681],
        [-3.26157671, -1.94930836, -1.06886462, ..., -1.01361335,
         -0.47764129, -0.22642321],
        ..., 
        [-2.504397  , -1.94930836,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [-2.443075  , -1.94930836,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [-2.3750976 , -1.94930836,  0.        , ...,  0.        ,
          0.        ,  0.        ]]])

In [18]:
val_target[:1]

array([[ 0.20840765,  0.20843489,  0.20848162,  0.20850593,  0.20846738,
         0.20832822,  0.20804722,  0.20760371,  0.20694196,  0.20606234,
         0.20490275,  0.20346545,  0.20170656,  0.19960787,  0.19715438,
         0.19430371,  0.19107213,  0.18739457,  0.18330397,  0.17875601,
         0.17376233,  0.16831048,  0.16239823,  0.15603843,  0.14922933,
         0.14199533,  0.13435532,  0.12633575,  0.11797613,  0.10931933,
         0.10040661,  0.09132636,  0.0820916 ,  0.07286793,  0.06364411,
         0.05459198,  0.04576255,  0.03728091,  0.02929574,  0.02184746,
         0.01521761,  0.00935695,  0.00460781,  0.00093816, -0.00140342,
        -0.00232609, -0.00176093,  0.00048043,  0.00423972,  0.0097068 ,
         0.0165194 ,  0.02454485,  0.03330231,  0.04224135,  0.05049569,
         0.05725584,  0.06100049,  0.06118098,  0.05643725,  0.04723103,
         0.03299822,  0.01284984, -0.01742873, -0.06424224, -0.05943314]])

In [26]:
def sliding_window(data, size, stepsize=1, padded=False, axis=-1, copy=True):
    """
    Calculate a sliding window over a signal
    Parameters
    ----------
    data : numpy array
        The array to be slided over.
    size : int
        The sliding window size
    stepsize : int
        The sliding window stepsize. Defaults to 1.
    axis : int
        The axis to slide over. Defaults to the last axis.
    copy : bool
        Return strided array as copy to avoid sideffects when manipulating the
        output array.
    Returns
    -------
    data : numpy array
        A matrix where row in last dimension consists of one instance
        of the sliding window.
    Notes
    -----
    - Be wary of setting `copy` to `False` as undesired sideffects with the
      output values may occurr.
    Examples
    --------
    >>> a = numpy.array([1, 2, 3, 4, 5])
    >>> sliding_window(a, size=3)
    array([[1, 2, 3],
           [2, 3, 4],
           [3, 4, 5]])
    >>> sliding_window(a, size=3, stepsize=2)
    array([[1, 2, 3],
           [3, 4, 5]])
    See Also
    --------
    pieces : Calculate number of pieces available by sliding
    """
    if axis >= data.ndim:
        raise ValueError(
            "Axis value out of range"
        )

    if stepsize < 1:
        raise ValueError(
            "Stepsize may not be zero or negative"
        )

    if size > data.shape[axis]:
        raise ValueError(
            "Sliding window size may not exceed size of selected axis"
        )

    shape = list(data.shape)
    shape[axis] = np.floor(data.shape[axis] / stepsize - size / stepsize + 1).astype(int)
    shape.append(size)

    strides = list(data.strides)
    strides[axis] *= stepsize
    strides.append(data.strides[axis])

    strided = np.lib.stride_tricks.as_strided(
        data, shape=shape, strides=strides
    )

    if copy:
        return strided.copy()
    else:
        return strided

In [32]:
def np_version():
    
    a = np.arange(10)
    return sliding_window(a, 3)
    

In [33]:
np_version()

array([[0, 1, 2],
       [1, 2, 3],
       [2, 3, 4],
       [3, 4, 5],
       [4, 5, 6],
       [5, 6, 7],
       [6, 7, 8],
       [7, 8, 9]])

In [42]:
def py_version():
    a = np.arange(1, 9)
    final = []
    for i in a:
        final.append([])
        for x in range(i-1, i+2):
            
            final[-1].append(x)
    return final

In [43]:
py_version()

[[0, 1, 2],
 [1, 2, 3],
 [2, 3, 4],
 [3, 4, 5],
 [4, 5, 6],
 [5, 6, 7],
 [6, 7, 8],
 [7, 8, 9]]